# About this notebook
- [Luke](https://arxiv.org/pdf/2010.01057v1.pdf)-base starter notebook
- [Training notebook](https://www.kaggle.com/yasufuminakama/jigsaw4-luke-base-starter-train)
- Approach References
    - https://www.kaggle.com/c/jigsaw-toxic-severity-rating/discussion/286471
    - https://www.kaggle.com/debarshichanda/pytorch-w-b-jigsaw-starter
    - https://www.kaggle.com/debarshichanda/0-816-jigsaw-inference
    - Thanks for sharing @debarshichanda

# Directory settings

In [ ]:
import numpy as np
import pandas as pd

df_val = pd.read_csv("../input/jigsaw-toxic-severity-rating/validation_data.csv")
print('less_toxic nuniques...', df_val.less_toxic.nunique())
print('more_toxic nuniques...', df_val.more_toxic.nunique())

aux1 = df_val[['less_toxic']].copy()
aux1.columns = ['text']
aux2 = df_val[['more_toxic']].copy()
aux2.columns = ['text']

df_val_comments = pd.concat([aux1,aux2],ignore_index=True)
print('val_comments nuniques...', df_val_comments.text.nunique())

df_val_comments = pd.DataFrame(data= df_val_comments.text.unique(),columns= ['text'])
print('val_comments nuniques...', len(df_val_comments))

df_val_comments.to_csv('df_val_comments.csv',index=False)

del aux1,aux2

In [ ]:
base = pd.read_csv('../input/jigsaw4-luke-base-training-tito-cv-strategy/oof_df.csv')
del base['fold']

val_comments = pd.read_csv('df_val_comments.csv')

val_comments = val_comments.merge(base[['less_toxic','less_toxic_pred']],left_on='text', right_on='less_toxic',how='left')
del val_comments['less_toxic']

val_comments = val_comments.merge(base[['more_toxic','more_toxic_pred']],left_on='text', right_on='more_toxic',how='left')
del val_comments['more_toxic']

val_comments.loc[val_comments.more_toxic_pred.isna()==True,'more_toxic_pred'] = \
                            val_comments.loc[val_comments.more_toxic_pred.isna()==True,'less_toxic_pred'] 

print(len(val_comments.loc[val_comments.more_toxic_pred.isna()==True,'more_toxic_pred']))

predictions = val_comments[['text','more_toxic_pred']].copy()
predictions.columns = ['text', 'base']

predictions = predictions.drop_duplicates(['text'])
predictions.index = range(len(predictions))

predictions

In [ ]:
large = pd.read_csv('../input/jigsaw4-luke-large-training-tito-cv-strategy/oof_df.csv')
del large['fold']

val_comments = pd.read_csv('df_val_comments.csv')

val_comments = val_comments.merge(large[['less_toxic','less_toxic_pred']],left_on='text', right_on='less_toxic',how='left')
del val_comments['less_toxic']

val_comments = val_comments.merge(large[['more_toxic','more_toxic_pred']],left_on='text', right_on='more_toxic',how='left')
del val_comments['more_toxic']

val_comments.loc[val_comments.more_toxic_pred.isna()==True,'more_toxic_pred'] = \
                            val_comments.loc[val_comments.more_toxic_pred.isna()==True,'less_toxic_pred'] 

print(len(val_comments.loc[val_comments.more_toxic_pred.isna()==True,'more_toxic_pred']))


predictions2 = val_comments[['text','more_toxic_pred']].copy()
predictions2.columns = ['text', 'large']

predictions2 = predictions2.drop_duplicates(['text'])
predictions2.index = range(len(predictions2))

predictions2



In [ ]:

predictions['large'] = predictions2['large']
del predictions2



In [ ]:
weight = []
result = []
for w in np.arange(0,1.01,0.01):
    val_data = pd.read_csv('../input/jigsaw-toxic-severity-rating/validation_data.csv')
    predictions['merging'] = w*predictions.base+(1-w)*predictions.large
    
    val_data = val_data.merge(predictions[['text','merging']],left_on='less_toxic', right_on='text',how='left')
    del val_data['text']
    val_data.columns = ['worker', 'less_toxic', 'more_toxic', 'less_toxic_predict']
    val_data = val_data.merge(predictions[['text','merging']],left_on='more_toxic', right_on='text',how='left')
    del val_data['text']    
    val_data.columns = ['worker', 'less_toxic', 'more_toxic', 'less_toxic_predict', 'more_toxic_predict']
    p1 = val_data.less_toxic_predict.values
    p2 = val_data.more_toxic_predict.values
    
    weight.append(w)
    result.append(np.round((p1 < p2).mean() * 100,2))
    
    print('Validation Accuracy with w:',w,'is...', np.round((p1 < p2).mean() * 100,2))
    


In [ ]:
best_weight = weight[np.argmax(result)]
best_result = max(result)


print(best_result)
print(best_weight)

predictions['merging'] = best_weight*predictions.base+(1-best_weight)*predictions.large
predictions.to_csv('predictions.csv',index=False)

predictions